In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from pesquisa_eleitoral.aas import AASs

In [2]:
pd.options.display.float_format = "{:_.0f}".format

In [3]:
DADOS: Path = Path(".").parent.resolve() / "data" / "raw"
AMOSTRAS: Path = Path(".").parent.resolve() / "data" / "sample"

In [4]:
erro_amostral = 0.02
significancia = 0.05

In [5]:
plano = AASs(erro_amostral=erro_amostral, significancia=significancia)

In [ ]:
dfs = []

for arquivo in DADOS.glob("*.csv"):
    cargo, turno, *_ = arquivo.stem.split("_")
    df = pd.read_csv(arquivo, index_col=0).assign(
        cargo=cargo, turno=int(turno)
    )
    dfs.append(df)

df = pd.concat(dfs)

In [13]:
tamanhos_amostrais = (
    df.loc[lambda df_: df_["cargo"] != "Presidente", :]
    .groupby(["estado", "turno", "cargo"], as_index=False)
    .count()
    .rename(columns={"nome": "N"})
    .assign(n=lambda df_: plano.tamanho_amostral(df_["N"]))
    .astype({"N": int, "n": int})
)
tamanhos_amostrais

,estado,turno,cargo,N,n
0,AC,1,Governador,455438,2389
1,AC,1,Senador,455438,2389
2,RO,1,Governador,925763,2395
3,RO,1,Senador,925763,2395
4,RO,2,Governador,925563,2395


In [10]:
random_state = 789

for (uf, turno, cargo), tamanho in tamanhos_amostrais.iterrows():
    sample = (
        df.query(
            f"estado == {uf!r} and turno == {turno} and cargo == {cargo!r}"
        )
        .sort_values(by="nome")
        .sample(tamanho.n, replace=False, random_state=random_state)
    )
    sample[["nome", "estado"]].to_csv(
        AMOSTRAS / f"{cargo}_{turno}_turno_{uf}.csv"
    )
    print(uf, turno, cargo, sample.shape[0])

AC 1 Governador 2389
AC 1 Senador 2389
RO 1 Governador 2395
RO 1 Senador 2395
RO 2 Governador 2395
